In [1]:
versioninfo()

Julia Version 1.4.0
Commit b8e9a9ecc6 (2020-03-21 16:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, skylake)


In [2]:
using Pkg
Pkg.activate("../..")
Pkg.status()

 Activating environment at `~/Dropbox/class/M1399.000200/2020/Project.toml`


Status `~/Dropbox/class/M1399.000200/2020/Project.toml`
  [6e4b80f9] BenchmarkTools v0.5.0
  [31c24e10] Distributions v0.23.10
  [c91e804a] Gadfly v1.3.0
  [bd48cda9] GraphRecipes v0.5.4
  [42fd0dbc] IterativeSolvers v0.8.4
  [b51810bb] MatrixDepot v0.8.0
  [47be7bcc] ORCA v0.5.0
  [a03496cd] PlotlyBase v0.4.1
  [f0f68f2c] PlotlyJS v0.14.0
  [91a5bcdd] Plots v1.6.1
  [438e738f] PyCall v1.91.4
  [d330b81b] PyPlot v2.9.0
  [6f49c342] RCall v0.13.7
  [2913bbd2] StatsBase v0.32.2
  [b8865327] UnicodePlots v1.3.0
  [8f399da3] Libdl 
  [10745b16] Statistics 


# Iterative Methods for Solving Linear Equations

## Introduction

So far we have considered direct methods for solving linear equations.    

* **Direct methods** (flops fixed _a priori_) vs **iterative methods**:
    - Direct method (GE/LU, Cholesky, QR, SVD): accurate. good for dense, small to moderate sized $\mathbf{A}$  
    - Iterative methods (Jacobi, Gauss-Seidal, SOR, conjugate-gradient, GMRES): accuracy depends on number of iterations. good for large, sparse, structured linear system, parallel computing, warm start (reasonable accuracy after, say, 100 iterations)

## Motivation: PageRank 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/PageRanks-Example.svg/400px-PageRanks-Example.svg.png" width="400" align="center"/>

Source: [Wikepedia](https://en.wikipedia.org/wiki/PageRank)

* $\mathbf{A}  \in \{0,1\}^{n \times n}$ the connectivity matrix of webpages with entries
$$
\begin{eqnarray*}
	a_{ij} = \begin{cases}
	1 &  \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}. 
\end{eqnarray*}
$$
$n \approx 10^9$ in May 2017.

* $r_i = \sum_j a_{ij}$ is the *out-degree* of page $i$. 

* [Larry Page](https://en.wikipedia.org/wiki/PageRank) imagined a random surfer wandering on internet according to following rules:
    - From a page $i$ with $r_i>0$
        * with probability $p$, (s)he randomly chooses a link $j$ from page $i$ (uniformly) and follows that link to the next page  
        * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page 
    - From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page
    
* The process defines an $n$-state Markov chain, where each state corresponds to each page. 
$$
    p_{ij} = (1-p)\frac{1}{n} + p\frac{a_{ij}}{r_i}
$$
with interpretation $a_{ij}/r_i = 1/n$ if $r_i = 0$.

* Stationary distribution of this Markov chain gives the score (long term probability of visit) of each page.

* Stationary distribution can be obtained as the top *left* eigenvector of the transition matrix $\mathbf{P}=(p_{ij})$ corresponding to eigenvalue 1. 
$$
    \mathbf{x}^T\mathbf{P} = \mathbf{x}^T.
$$
Equivalently it can be cast as a linear equation.
$$
    (\mathbf{I} - \mathbf{P}^T) \mathbf{x} = \mathbf{0}.
$$

* You've got to solve a linear equation with $10^9$ variables!

* GE/LU will take $2 \times (10^9)^3/3/10^{12} \approx 6.66 \times 10^{14}$ seconds $\approx 20$ million years on a tera-flop supercomputer!

* Iterative methods come to the rescue.

## Splitting and fixed point iteration

* The key idea of iterative method for solving $\mathbf{A}\mathbf{x}=\mathbf{b}$ is to **split** the matrix $\mathbf{A}$ so that
$$
    \mathbf{A} = \mathbf{M} - \mathbf{K}
$$
where $\mathbf{M}$ is invertible and easy to invert.

* Then $\mathbf{A}\mathbf{x} = \mathbf{M}\mathbf{x} - \mathbf{K}\mathbf{x} = \mathbf{b}$ or
$$
    \mathbf{x} = \mathbf{M}^{-1}\mathbf{K}\mathbf{x} - \mathbf{M}^{-1}\mathbf{b}
    .
$$
Thus a solution to $\mathbf{A}\mathbf{x}=\mathbf{b}$ is a fixed point of iteration
$$
    \mathbf{x}^{(t+1)} = \mathbf{M}^{-1}\mathbf{K}\mathbf{x}^{(t)} - \mathbf{M}^{-1}\mathbf{b}
    = \mathbf{R}\mathbf{x}^{(k)} - \mathbf{c}
    .
$$

* Under a suitable choice of $\mathbf{R}$, i.e., splitting of $\mathbf{A}$, the sequence $\mathbf{x}^{(k)}$ generated by the above iteration converges to a solution $\mathbf{A}\mathbf{x}=\mathbf{b}$:

> Let $\rho(\mathbf{R})=\max_{i=1,\dotsc,n}|\lambda_i(R)|$, where $\lambda_i(R)$ is the $i$th (complex) eigenvalue of $\mathbf{R}$. The iteration $\mathbf{x}^{(t+1)}=\mathbf{R}\mathbf{x}^{(t)} - \mathbf{c}$ converges to a solution to $\mathbf{A}\mathbf{x}=\mathbf{b}$ if and only if $\rho(\mathbf{R}) < 1$.

## Jacobi's method

$$
x_i^{(t+1)} = \frac{b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(t)} - \sum_{j=i+1}^n a_{ij} x_j^{(t)}}{a_{ii}}.
$$


* Split $\mathbf{A} = \mathbf{L} + \mathbf{D} + \mathbf{U}$ = (strictly lower triangular) + (diagonal) + (strictly upper triangular).


* Take $\mathbf{M}=\mathbf{D}$ (easy to invert!) and $\mathbf{K}=-(\mathbf{L} + \mathbf{U})$:
$$
    \mathbf{D} \mathbf{x}^{(t+1)} = - (\mathbf{L} + \mathbf{U}) \mathbf{x}^{(t)} + \mathbf{b},
$$
i.e., 
$$
	\mathbf{x}^{(t+1)} = - \mathbf{D}^{-1} (\mathbf{L} + \mathbf{U}) \mathbf{x}^{(t)} + \mathbf{D}^{-1} \mathbf{b} = - \mathbf{D}^{-1} \mathbf{A} \mathbf{x}^{(t)} + \mathbf{x}^{(t)} + \mathbf{D}^{-1} \mathbf{b}.
$$

* Convergence is guaranteed if $\mathbf{A}$ is striclty row diagonally dominant: $|a_{ii}| > \sum_{j\neq i}|a_{ij}|$.

* One round costs $2n^2$ flops with an **unstructured** $\mathbf{A}$. Gain over GE/LU if converges in $o(n)$ iterations. 

* Saving is huge for **sparse** or **structured** $\mathbf{A}$. By structured, we mean the matrix-vector multiplication $\mathbf{A} \mathbf{v}$ is fast ($O(n)$ or $O(n \log n)$).
    - Often the multiplication is implicit and $\mathbf{A}$ is not even stored, e.g., finite difference: $(\mathbf{A}\mathbf{v})_i = v_i - v_{i+1}$.

## Gauss-Seidel method

$$
x_i^{(t+1)} = \frac{b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(t+1)} - \sum_{j=i+1}^n a_{ij} x_j^{(t)}}{a_{ii}}.
$$

* Split $\mathbf{A} = \mathbf{L} + \mathbf{D} + \mathbf{U}$ = (strictly lower triangular) + (diagonal) + (strictly upper triangular) as Jacobi.

* Take $\mathbf{M}=\mathbf{D}+\mathbf{L}$ (easy to invert, why?) and $\mathbf{K}=-\mathbf{U}$:
$$
(\mathbf{D} + \mathbf{L}) \mathbf{x}^{(t+1)} = - \mathbf{U} \mathbf{x}^{(t)} + \mathbf{b}
$$
i.e., 
$$
\mathbf{x}^{(t+1)} = - (\mathbf{D} + \mathbf{L})^{-1} \mathbf{U} \mathbf{x}^{(t)} + (\mathbf{D} + \mathbf{L})^{-1} \mathbf{b}.
$$

* Equivalent to
$$
\mathbf{D}\mathbf{x}^{(t+1)} = - \mathbf{L} \mathbf{x}^{(t+1)} - \mathbf{U} \mathbf{x}^{(t)} + \mathbf{b}
$$
or
$$
\mathbf{x}^{(t+1)} = \mathbf{D}^{-1}(- \mathbf{L} \mathbf{x}^{(t+1)} - \mathbf{U} \mathbf{x}^{(t)} + \mathbf{b})
$$
leading to the iteration.

* "Coordinate descent" version of Jacobi.

* Convergence is guaranteed if $\mathbf{A}$ is striclty row diagonally dominant.

## Successive over-relaxation (SOR)

$$
x_i^{(t+1)} = \frac{\omega}{a_{ii}} \left( b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(t+1)} - \sum_{j=i+1}^n a_{ij} x_j^{(t)} \right)+ (1-\omega) x_i^{(t)},
$$

* $\omega=1$: Gauss-Seidel; $\omega \in (0, 1)$: underrelaxation; $\omega > 1$: overrelaxation
    - Relaxation in hope of faster convergence
    
* Split $\mathbf{A} = \mathbf{L} + \mathbf{D} + \mathbf{U}$ = (strictly lower triangular) + (diagonal) + (strictly upper triangular) as before.

* Take $\mathbf{M}=\frac{1}{\omega}\mathbf{D}+\mathbf{L}$ (easy to invert, why?) and $\mathbf{K}=\frac{1-\omega}{\omega}\mathbf{D}-\mathbf{U}$:
$$
\begin{split}
(\mathbf{D} + \omega \mathbf{L})\mathbf{x}^{(t+1)} &= [(1-\omega) \mathbf{D} - \omega \mathbf{U}] \mathbf{x}^{(t)} +  \omega \mathbf{b} 
\\
\mathbf{D}\mathbf{x}^{(t+1)} &= (1-\omega) \mathbf{D}\mathbf{x}^{(t)} + \omega ( -\mathbf{U}\mathbf{x}^{(t)} - \mathbf{L}\mathbf{x}^{(t+1)}  + \mathbf{b} )
\\
\mathbf{x}^{(t+1)} &= (1-\omega)\mathbf{x}^{(t)} + \omega \mathbf{D}^{-1} ( -\mathbf{U}\mathbf{x}^{(t)} - \mathbf{L}\mathbf{x}^{(t+1)}  + \mathbf{b} )
\end{split}
$$

## Conjugate gradient method

* **Conjugate gradient and its variants are the top-notch iterative methods for solving huge, structured linear systems.**

* Key idea: solving $\mathbf{A} \mathbf{x} = \mathbf{b}$ is equivalent to minimizing the quadratic function $\frac{1}{2} \mathbf{x}^T \mathbf{A} \mathbf{x} - \mathbf{b}^T \mathbf{x}$ if $\mathbf{A}$ *is positive definite*.

[Application to a fusion problem in physics](http://www.sciencedirect.com/science/article/pii/0021999178900980?via%3Dihub):

| Method                                 | Number of Iterations |
|----------------------------------------|----------------------|
| Gauss Seidel                           | 208,000              |
| Block SOR methods                      | 765                  |
| Incomplete Cholesky conjugate gradient | 25                   |


* More on CG later.

## MatrixDepot.jl

MatrixDepot.jl is an extensive collection of test matrices in Julia. After installation, we can check available test matrices by

In [3]:
using MatrixDepot

mdinfo()

include group.jl for user defined matrix generators
verify download of index files...
used remote site is https://sparse.tamu.edu/?per_page=All
populating internal database...


### Currently loaded Matrices

| builtin(#)  |             |              |             |               |
|:----------- |:----------- |:------------ |:----------- |:------------- |
| 1 baart     | 13 fiedler  | 25 invhilb   | 37 parter   | 49 shaw       |
| 2 binomial  | 14 forsythe | 26 invol     | 38 pascal   | 50 smallworld |
| 3 blur      | 15 foxgood  | 27 kahan     | 39 pei      | 51 spikes     |
| 4 cauchy    | 16 frank    | 28 kms       | 40 phillips | 52 toeplitz   |
| 5 chebspec  | 17 gilbert  | 29 lehmer    | 41 poisson  | 53 tridiag    |
| 6 chow      | 18 golub    | 30 lotkin    | 42 prolate  | 54 triw       |
| 7 circul    | 19 gravity  | 31 magic     | 43 randcorr | 55 ursell     |
| 8 clement   | 20 grcar    | 32 minij     | 44 rando    | 56 vand       |
| 9 companion | 21 hadamard | 33 moler     | 45 randsvd  | 57 wathen     |
| 10 deriv2   | 22 hankel   | 34 neumann   | 46 rohess   | 58 wilkinson  |
| 11 dingdong | 23 heat     | 35 oscillate | 47 rosser   | 59 wing       |
| 12 erdrey   | 24 hilb     | 36 parallax  | 48 sampling |               |

| user(#) |
|:------- |

| Groups  |       |       |         |        |         |           |     |     |     |     |     |
|:------- |:----- |:----- |:------- |:------ |:------- |:--------- |:--- |:--- |:--- |:--- |:--- |
| all     | local | eigen | illcond | posdef | regprob | symmetric |     |     |     |     |     |
| builtin | user  | graph | inverse | random | sparse  |           |     |     |     |     |     |

| Suite Sparse | of   |
|:------------ |:---- |
| 0            | 2833 |

| MatrixMarket | of  |
|:------------ |:--- |
| 0            | 498 |


In [4]:
# List matrices that are positive definite and sparse:
mdlist(:symmetric & :posdef & :sparse)

2-element Array{String,1}:
 "poisson"
 "wathen"

In [5]:
# Get help on Poisson matrix
mdinfo("poisson")

# Poisson Matrix (poisson)

A block tridiagonal matrix from Poisson’s equation.      This matrix is sparse, symmetric positive definite and      has known eigenvalues. The result is of type `SparseMatrixCSC`.

*Input options:*

  * [type,] dim: the dimension of the matirx is `dim^2`.

*Groups:* ["inverse", "symmetric", "posdef", "eigen", "sparse"]

*References:*

**G. H. Golub and C. F. Van Loan**, Matrix Computations,           second edition, Johns Hopkins University Press, Baltimore,           Maryland, 1989 (Section 4.5.4).


In [6]:
# Generate a Poisson matrix of dimension n=10
A = matrixdepot("poisson", 10)

100×100 SparseArrays.SparseMatrixCSC{Float64,Int64} with 460 stored entries:
  [1  ,   1]  =  4.0
  [2  ,   1]  =  -1.0
  [11 ,   1]  =  -1.0
  [1  ,   2]  =  -1.0
  [2  ,   2]  =  4.0
  [3  ,   2]  =  -1.0
  [12 ,   2]  =  -1.0
  [2  ,   3]  =  -1.0
  [3  ,   3]  =  4.0
  [4  ,   3]  =  -1.0
  [13 ,   3]  =  -1.0
  [3  ,   4]  =  -1.0
  ⋮
  [97 ,  97]  =  4.0
  [98 ,  97]  =  -1.0
  [88 ,  98]  =  -1.0
  [97 ,  98]  =  -1.0
  [98 ,  98]  =  4.0
  [99 ,  98]  =  -1.0
  [89 ,  99]  =  -1.0
  [98 ,  99]  =  -1.0
  [99 ,  99]  =  4.0
  [100,  99]  =  -1.0
  [90 , 100]  =  -1.0
  [99 , 100]  =  -1.0
  [100, 100]  =  4.0

In [7]:
using UnicodePlots
spy(A)

                    Sparsity Pattern
       ┌──────────────────────────────────────────┐    
     1 │⠻⣦⡀⠀⠘⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
       │⠀⠈⠻⣦⡀⠀⠉⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
       │⠒⢄⠀⠈⠱⣦⡀⠀⠑⢢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠣⢄⠀⠈⠻⣦⡀⠀⠑⠤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠱⣀⠀⠈⠱⣦⡀⠀⠑⢄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠑⡄⠀⠈⠻⣦⡀⠀⠘⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠈⠑⢄⠀⠈⠱⣦⡀⠀⠉⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠒⢄⠀⠈⠻⣦⡀⠀⠑⢢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠣⢄⠀⠈⠛⣤⡀⠀⠑⠤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠱⣀⠀⠈⠻⣦⡀⠀⠑⢄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⡄⠀⠈⠛⣤⡀⠀⠘⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠑⢄⠀⠈⠻⣦⡀⠀⠉⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠒⢄⠀⠈⠛⣤⡀⠀⠑⢢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠣⢄⠀⠈⠻⣦⡀⠀⠑⠤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠱⣀⠀⠈⠻⢆⡀⠀⠑⢄⡀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⡄⠀⠈⠻⣦⡀⠀⠘⢄⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀

In [8]:
# Get help on Wathen matrix
mdinfo("wathen")

# Wathen Matrix (wathen)

Wathen Matrix is a sparse, symmetric positive, random matrix arose from the finite element method. The generated matrix is the consistent mass matrix for a regular nx-by-ny grid of 8-nodes.

*Input options:*

  * [type,] nx, ny: the dimension of the matrix is equal to   `3 * nx * ny + 2 * nx * ny + 1`.
  * [type,] n: `nx = ny = n`.

*Groups:* ["symmetric", "posdef", "eigen", "random", "sparse"]

*References:*

**A. J. Wathen**, Realistic eigenvalue bounds for     the Galerkin mass matrix, IMA J. Numer. Anal., 7 (1987),     pp. 449-457.


In [9]:
# Generate a Wathen matrix of dimension n=5
A = matrixdepot("wathen", 5)

96×96 SparseArrays.SparseMatrixCSC{Float64,Int64} with 1256 stored entries:
  [1 ,  1]  =  10.5349
  [2 ,  1]  =  -10.5349
  [3 ,  1]  =  3.51163
  [12,  1]  =  -10.5349
  [13,  1]  =  -14.0465
  [18,  1]  =  3.51163
  [19,  1]  =  -14.0465
  [20,  1]  =  5.26744
  [1 ,  2]  =  -10.5349
  [2 ,  2]  =  56.186
  [3 ,  2]  =  -10.5349
  [12,  2]  =  35.1163
  ⋮
  [84, 95]  =  41.4051
  [85, 95]  =  41.4051
  [94, 95]  =  -12.4215
  [95, 95]  =  66.2481
  [96, 95]  =  -12.4215
  [77, 96]  =  6.21076
  [78, 96]  =  -16.562
  [79, 96]  =  4.14051
  [84, 96]  =  -16.562
  [85, 96]  =  -12.4215
  [94, 96]  =  4.14051
  [95, 96]  =  -12.4215
  [96, 96]  =  12.4215

In [10]:
spy(A)

                   Sparsity Pattern
      ┌──────────────────────────────────────────┐    
    1 │⠿⣧⡄⠀⠀⢿⡄⠸⢿⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
      │⠀⠉⠿⣧⣀⠈⣧⡀⠈⠹⣧⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
      │⣤⣄⡀⠘⠛⣤⡘⢣⣤⣀⠀⠛⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⣀⡉⠉⠻⠶⣈⠻⢆⣈⠉⠛⠷⢆⠀⠀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠛⣷⣆⡀⠀⢻⡆⠘⢻⣶⡀⠀⢸⣆⠀⠛⣷⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠉⢿⣤⠀⢿⡄⠀⠈⠿⣧⡄⠹⣧⠀⠈⠹⢿⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠈⠉⠀⠈⠑⠲⢶⣄⡉⠱⣦⡉⠒⢶⣤⣈⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⢠⣤⠀⠉⠛⢣⠈⠛⣤⡄⠈⠙⢣⡄⠀⢠⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠈⠙⢻⣆⡀⠘⣷⡀⠉⢻⣶⣀⠈⢻⣆⠈⠛⣷⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⠷⠆⠘⠷⣀⡀⠘⠻⢆⡀⠻⢆⡀⠀⠻⠶⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⢶⣤⡈⠻⣦⡈⠛⠷⣦⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠶⣦⠀⠈⠱⣦⠈⠱⣦⡄⠈⠉⢶⡄⠰⢶⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠹⢿⣤⡀⠹⣧⡀⠉⠿⣧⣀⠈⢿⡄⠈⠹⣧⣄⡀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠃⠀⠘⢣⣄⡀⠘⠛⣤⡀⢣⣤⣀⠀⠛⠃⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡉⠛⠷⠤⣈⠻⢆⣈⠙⠷⠦⢄⡀⠀⣀⡀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⣷⣆⡀⠀⢻⣆⠘⢻⣶⡀⠀⠘⣷⠀⠛⣷⣀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

## Numerical examples

The [`IterativeSolvers.jl`](https://github.com/JuliaMath/IterativeSolvers.jl) package implements most commonly used iterative solvers.

### Generate test matrix

In [ ]:
using BenchmarkTools, IterativeSolvers, LinearAlgebra, MatrixDepot, Random

Random.seed!(280)

n = 100
# Poisson matrix of dimension n^2=10000, pd and sparse
A = matrixdepot("poisson", n)
@show typeof(A)
# dense matrix representation of A
Afull = convert(Matrix, A)
@show typeof(Afull)
# sparsity level
count(!iszero, A) / length(A)

In [ ]:
spy(A)

In [ ]:
# storage difference
Base.summarysize(A), Base.summarysize(Afull)

### Matrix-vector muliplication

In [ ]:
# randomly generated vector of length n^2
b = randn(n^2)
# dense matrix-vector multiplication
@benchmark $Afull * $b

In [ ]:
# sparse matrix-vector multiplication
@benchmark $A * $b

### Dense solve via Cholesky

In [ ]:
# record the Cholesky solution
xchol = cholesky(Afull) \ b;

In [ ]:
# dense solve via Cholesky
@benchmark cholesky($Afull) \ $b

### Jacobi solver

It seems that Jacobi solver doesn't give the correct answer.

In [ ]:
xjacobi = jacobi(A, b)
@show norm(xjacobi - xchol)

[Documentation](https://juliamath.github.io/IterativeSolvers.jl/dev/linear_systems/stationary/#Jacobi-1) reveals that the default value of `maxiter` is 10. A couple trial runs shows that 30000 Jacobi iterations are enough to get an accurate solution.

In [ ]:
xjacobi = jacobi(A, b, maxiter=30000)
@show norm(xjacobi - xchol)

In [ ]:
@benchmark jacobi($A, $b, maxiter=30000)

### Gauss-Seidal

In [ ]:
# Gauss-Seidel solution is fairly close to Cholesky solution after 15000 iters
xgs = gauss_seidel(A, b, maxiter=15000)
@show norm(xgs - xchol)

In [ ]:
@benchmark gauss_seidel($A, $b, maxiter=15000)

### SOR

In [ ]:
# symmetric SOR with ω=0.75
xsor = ssor(A, b, 0.75, maxiter=10000)
@show norm(xsor - xchol)

In [ ]:
@benchmark sor($A, $b, 0.75, maxiter=10000)

### Conjugate Gradient

In [ ]:
# conjugate gradient
xcg = cg(A, b)
@show norm(xcg - xchol)

In [ ]:
@benchmark cg($A, $b)

* A basic tenet in numerical analysis: 

> **The structure should be exploited whenever possible in solving a problem.** 

## Acknowledgment

Many parts of this lecture note is based on [Dr. Hua Zhou](http://hua-zhou.github.io)'s 2019 Spring Statistical Computing course notes available at <http://hua-zhou.github.io/teaching/biostatm280-2019spring/index.html>.